In [2]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [4]:
import os
import logging
from dotenv import load_dotenv
from langchain.embeddings.base import Embeddings
import bs4
from langchain_core.vectorstores import InMemoryVectorStore

import asyncio

load_dotenv()

# Embeddings service using Azure OpenAI API
class EmbeddingsService:
    def __init__(self):
        from openai import AzureOpenAI
        self.client = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("API_VERSION")
        )
        self.model_name = os.getenv("MODEL_NAME")

    async def get_embeddings(self, text: str):
        try:
            response = self.client.embeddings.create(input=text, model="embedding")
            result = response.data[0].embedding
            logging.info(f"Full embedding: {result}")
            logging.info(f"dimension: {len(result)}")
            return result
        except Exception as e:
            logging.error(f"Error generating embeddings: {str(e)}")
            return []

class CustomAzureOpenAIEmbeddings(Embeddings):
    def __init__(self, embedding_service: EmbeddingsService):
        self.embedding_service = embedding_service

    def embed_query(self, text: str):
        # Use asyncio.run to get synchronous embeddings for a single query
        result = asyncio.run(self.embedding_service.get_embeddings(text))
        return result

    def embed_documents(self, texts: list):
        # Run each async embedding call synchronously
        results = [asyncio.run(self.embedding_service.get_embeddings(text)) for text in texts]
        return results
    
# Initialize the custom embedding service and pass it into the RAG setup
embedding_service = EmbeddingsService()
custom_embeddings = CustomAzureOpenAIEmbeddings(embedding_service)


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# Load the document, split it into chunks, and embed each chunk.
loader = TextLoader("corpus.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = custom_embeddings

Created a chunk of size 2404, which is longer than the specified 300
Created a chunk of size 951, which is longer than the specified 300


In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from langchain_pinecone import PineconeVectorStore


index_name = "arctest"

import asyncio

async def initialize_pinecone_index():
    # Assume this is where PineconeVectorStore interacts with asyncio
    docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)
    return docsearch

# Run the function in the existing event loop
docsearch = await initialize_pinecone_index()

In [11]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

NameError: name 'db' is not defined